In [1]:
import numpy as np
from src.river import River

env = River()

In [2]:

def lrtdp(max_iter=1000, epsilon=.1, gamma=.9):
    v = np.zeros(env.S.size)
    H = np.zeros(env.S.size)
    solved = [False for _ in env.S]

    curr_iter = 0

    bell = lambda s:  max([sum([env.T(s,a,s_)*(env.R(s,a,s_) + gamma*v[s_]) for s_ in env.S]) for a in env.A])
    greedy = lambda s: np.argmax([sum([env.T(s,a,s_)*(env.R(s,a,s_) + gamma*v[s_]) for s_ in env.S]) for a in env.A])

    def check_solved(s):
        rv = True
        opened, closed = [], []
        if not solved[s]:
            opened.append(s)
        while len(opened)>0:
            s = opened.pop()
            closed.append(s)
            if abs(bell(s)-v[s]) > epsilon:
                rv = False
                continue
            a = greedy(s)
            for s_ in env.S:
                if env.T(s,a,s_)>0:
                    if not solved[s_] and not(s_ in opened+closed):
                        opened.append(s_)
        if rv:
            for s_ in closed:
                if not solved[s_]:
                    solved[s_] = True
        else:
            for s_ in closed:
                v[s_] = bell(s_)
        return rv

    while not(all([solved[s0] for s0 in env.S0])) and curr_iter<max_iter:
        curr_iter += 1
        visited = []
        s = np.random.choice(env.S0)
        while not solved[s]:
            visited.append(s)
            if s in env.G: break
            a = greedy(s)
            v[s] = bell(s)
            s = np.random.choice(env.S, p=[env.T(s,a,s_) for s_ in env.S])
        while len(visited) > 0:
            s = visited.pop()
            if not check_solved(s): break

    env.plot(list(np.round(v, 2)))
    return [greedy(s) for s in env.S]
        

pi = lrtdp(epsilon=.1, gamma=1)
env.plot(pi, True)

 _____________________________ 
| -5.0| -4.0| -3.0| -2.0| -1.0|
|_____|_____|_____|_____|_____|
| -6.0|-5.88|-3.75| -3.0| 0.0 |
|_____|_____|_____|_____|_____|

 _____________________________ 
|  →  |  →  |  →  |  →  |  ↓  |
|_____|_____|_____|_____|_____|
|  ↑  |  →  |  ↑  |  →  |  ↑  |
|_____|_____|_____|_____|_____|



In [3]:
pi

[0, 2, 0, 2, 0, 2, 2, 2, 2, 1]